In [7]:
# Directly accessing gmail api

import os.path

# !pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def gmail_authenticate():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is created
    # automatically when the authorization flow completes for the first time.
    if os.path.exists("token.json"):
        print('using exisiting token')
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("token expired - authentication required")
            creds.refresh(Request())
        else:
            print("authentication required")
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    # return the gmail service
    return build("gmail", "v1", credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

def get_messages(query):
    try:
        # Call the Gmail API
        result = service.users().messages().list(userId='me',q=query).execute()
        messages = [ ]
        if 'messages' in result:
            messages.extend(result['messages'])
        while 'nextPageToken' in result:
            page_token = result['nextPageToken']
            result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
            if 'messages' in result:
                messages.extend(result['messages'])
        return messages

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

# all messages
messages = get_messages(query='')
messages

using exisiting token


[{'id': '1915811b7ee07169', 'threadId': '1915811b7ee07169'},
 {'id': '1915803921e78d8c', 'threadId': '1915803921e78d8c'},
 {'id': '19157d6ab53a858a', 'threadId': '19157848c0a84e29'},
 {'id': '19157d6482df0d59', 'threadId': '19156915b38ba9d9'},
 {'id': '19157848c0a84e29', 'threadId': '19157848c0a84e29'},
 {'id': '191572115d90548b', 'threadId': '191572115d90548b'},
 {'id': '19156915b38ba9d9', 'threadId': '19156915b38ba9d9'},
 {'id': '1915669a36238f6c', 'threadId': '1915669a36238f6c'},
 {'id': '191564a03a12b00d', 'threadId': '191564a03a12b00d'},
 {'id': '191561a2fc367807', 'threadId': '191561a2fc367807'},
 {'id': '19155ab1d822fa1c', 'threadId': '19155ab1d822fa1c'},
 {'id': '191548c10932cc96', 'threadId': '191548c10932cc96'},
 {'id': '191544194b06aff9', 'threadId': '191544194b06aff9'},
 {'id': '191542d0b36b3ffb', 'threadId': '191542d0b36b3ffb'},
 {'id': '191542b5bfc5778f', 'threadId': '191542b5bfc5778f'},
 {'id': '19153f5817cfe57e', 'threadId': '19153f5817cfe57e'},
 {'id': '1915387f1f14317

In [8]:
# Using simplegmail abstraction

# !pip3 install simplegmail
from simplegmail import Gmail

gmail = Gmail()

# list lables
gmail.list_labels()

# Unread messages in your inbox
# messages = gmail.get_unread_inbox()

# Starred messages
messages = gmail.get_starred_messages()

# ...and many more easy to use functions can be found in gmail.py!

# Print them out!
for message in messages:
    # print("To: " + message.recipient)
    print("From: " + message.sender)
    print("Date: " + message.date)
    print("\tSubject: " + message.subject)
    # print("Preview: " + message.snippet)
    
    # print("Message Body: " + message.plain)  # or message.html

From: Skydive Space Center <skydivespacecenter@attractionsuite.com>
Date: 2024-08-09 11:34:50-04:00
	Subject: Your Online Reservation
From: Auto-Receipt <noreply@mail.authorize.net>
Date: 2024-08-09 11:34:45-04:00
	Subject: Transaction Receipt from Skydive Space Center for $100.00 (USD)
From: Jennifer Leale <jleale@fsu.edu>
Date: 2024-08-09 09:16:54-04:00
	Subject: Interview for the Application Developer/Designer Position
From: Google <google-noreply@google.com>
Date: 2024-07-28 11:52:25-04:00
	Subject: Your Gmail storage is 82% full


In [ ]:
# Get mails with query
from simplegmail import query

gmail.get_messages(query=query.construct_query(newer_than=(1, "day"), labels=[["INBOX"]]))